Test if jupyter notebook is runing on GPU

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


**Importer les bibs**

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2 as cv
import random
import math
import os

ModuleNotFoundError: No module named 'matplotlib'

Testing if jupyter notebook is running on GPU

In [2]:
print("Num GPUs available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs available:  0


In [3]:
tf.test.is_built_with_cuda()

True

In [13]:
print(tf.version.VERSION)

2.8.0


In [14]:
import sys
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

**Extraire les images de la dataset**

In [3]:
train_directory="C:/Users/ASUS/Desktop/train"
test_directory="C:/Users/ASUS/Desktop/test"

X_train=tf.keras.preprocessing.image_dataset_from_directory(train_directory, labels='inferred',image_size=(224, 224))
X_test=tf.keras.preprocessing.image_dataset_from_directory(test_directory, labels='inferred',image_size=(48, 48))

normalization_layer = tf.keras.layers.Rescaling(1./255)
X_train = X_train.map(lambda x, y: (normalization_layer(x), y)) 
X_test = X_test.map(lambda x, y: (normalization_layer(x), y)) 
normalization_layer=[]
AUTOTUNE = tf.data.AUTOTUNE
X_train = X_train.cache().prefetch(buffer_size=AUTOTUNE)
X_test = X_test.cache().prefetch(buffer_size=AUTOTUNE)

train_directory=[]

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [4]:
IMG_SHAPE =(224, 224, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False,weights='imagenet')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.layers[-1].output)
final_output = tf.keras.layers.Dense(128,activation='relu')(global_average_layer)
final_output = tf.keras.layers.Dense(64,activation='relu')(final_output)
model_output = tf.keras.layers.Dense(7, activation='softmax')(final_output)
model_input = base_model.input
model = tf.keras.Model(inputs=model_input, outputs=model_output)

9420800/9406464 [==============================] - 28s 3us/step


**Entrainnement**

In [ ]:
model.compile(
    optimizer="adam",
    loss='sparse_categorical_crossentropy',
    metrics=['acc'])

history = model.fit(X_train, 
                    epochs=30,
                    )

Epoch 1/30
 15/898 [..............................] - ETA: 1:09:09 - loss: 1.8186 - acc: 0.2958

**Evaluation du model**

In [ ]:
model.evaluate(X_test)

**Dessiner la courbe de train epochs/acc**

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
hist = history.history
plt.plot(hist["acc"])
plt.title("Accuracy plot")
plt.legend(["train"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("C:/Users/ASUS/Desktop/mobilenetv2_accuracy_e30_095.png")

**Sauvgarder le model**

In [ ]:
model.save('C:/Users/ASUS/Desktop/myModel_mobilenetv2_30_095.h5')

**Charger le model**

In [ ]:
model = tf.keras.models.load_model('C:/Users/ASUS/Desktop/myModel_1.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

**Activer la camera**

In [ ]:
emotions=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
cap=cv.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,100)
while True:
    suc,img = cap.read()
    
    # rgb to grayscale
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # appel haar-cascade 
    haar_cascade = cv.CascadeClassifier('.../.../haarcascade_frontalface_default.xml')

    # appliquer haar_cascade sur gray img
    faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9)

    # dessiner un rects sur le visage
    for (x, y, w, h) in faces_rect:
        cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    try:
        face=cv.resize(img[y:y+h,x:x+w],(224,224))
        result = model.predict(face[np.newaxis, ...])
        cv.putText(img, emotions[np.argmax(result)], (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 1,cv.LINE_AA)

    except:
        pass
    cv.imshow("emotion detector",img)
    
    if cv.waitKey(1)==ord('q'):
        break